# Exercise 2 - Transfer Learning VGG16 

In [180]:
import numpy as np
import keras
from keras.layers import Dense
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array, array_to_img
import tensorflow.image as tf_image
import matplotlib.pyplot as plt

Load in data, we will limit the dataset sizes for examples sake.

In [181]:
TRAIN_SIZE = 1000
TEST_SIZE = 5000

In [182]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [183]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

Preprocess to rgb

In [184]:
def preprocess(images):
    """ Preprocess grey scale image to rgb and resizes
    
    Args:
        images (np.array): Array of grey scale image
        
    Returns:
        np.array: Array of converted images
    """
    image_rgb = np.dstack((images, np.zeros_like(images), np.zeros_like(images)))
    image_rgb = image_rgb.reshape(image_rgb.shape[0], 28, 28, 3)
    image_rgb = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in image_rgb])
    return image_rgb
    
x_train_rgb = preprocess(x_train)
y_train_oh = to_categorical(y_train)

x_test_rgb = preprocess(x_test)
y_test_oh = to_categorical(y_test)

Extract features from VGG16

In [186]:
x_train_rgb.shape
train_features = model.predict(x_train_rgb[:TRAIN_SIZE])
train_features.shape

(1000, 1, 1, 512)

Train simple mlp on the extracted features

In [188]:
from keras.layers import Dense, Activation
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers

shape = np.array(train_features.shape)
train_features = np.reshape(train_features, (shape[0], shape[-1]))
print(train_features.shape)

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=512))
model.add(Dense(500, activation='sigmoid'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# fitting the model 

model.fit(train_features, y_train_oh[:TRAIN_SIZE], epochs=20, batch_size=128)

(1000, 512)
Epoch 1/20
1000/1000 [==============================] - 1s 834us/step - loss: 1.5923 - accuracy: 0.5220
Epoch 2/20
1000/1000 [==============================] - 0s 126us/step - loss: 0.7892 - accuracy: 0.7750
Epoch 3/20
1000/1000 [==============================] - 0s 130us/step - loss: 0.5353 - accuracy: 0.8510
Epoch 4/20
1000/1000 [==============================] - 0s 120us/step - loss: 0.4116 - accuracy: 0.8850
Epoch 5/20
1000/1000 [==============================] - 0s 121us/step - loss: 0.3170 - accuracy: 0.9140
Epoch 6/20
1000/1000 [==============================] - 0s 122us/step - loss: 0.2256 - accuracy: 0.9550
Epoch 7/20
1000/1000 [==============================] - 0s 121us/step - loss: 0.1717 - accuracy: 0.9710
Epoch 8/20
1000/1000 [==============================] - 0s 129us/step - loss: 0.1344 - accuracy: 0.9840
Epoch 9/20
1000/1000 [==============================] - 0s 122us/step - loss: 0.0978 - accuracy: 0.9890
Epoch 10/20
1000/1000 [=============================